In [7]:
def get(nums, i):
	if i == -1 or i == len(nums):
		return -float('inf')
	else:
		return nums[i]
def find_top(nums):
    if len(nums) == 0:
        return -1
    if len(nums) == 1:
        return nums[0]
    l, r = 0, len(nums) - 1
    while l <= r:
        mid = (l + r) // 2  # mid = l + (r – l) // 2
        if get(nums, mid - 1) < get(nums, mid) > get(nums, mid + 1):
            return mid
        if get(nums, mid - 1) > get(nums, mid):
            r = mid - 1
        elif get(nums, mid + 1) > get(nums, mid):
            l = mid + 1


In [5]:
nums = [3, 1, 2, 4]
find_top(nums)

0

In [1]:
from run_llama_new import LlamaDataset, create_data
import argparse
from tokenizer import Tokenizer
import torch
import json
from types import SimpleNamespace
from torch.utils.data import DataLoader
from classifier import LlamaZeroShotClassifier
import numpy as np

def get_args():
	parser = argparse.ArgumentParser()
	parser.add_argument("--train", type=str, default="data/cfimdb-train.txt")
	parser.add_argument("--dev", type=str, default="data/cfimdb-dev.txt")
	parser.add_argument("--test", type=str, default="data/cfimdb-test.txt")
	parser.add_argument("--label-names", type=str, default="data/cfimdb-label-mapping.json")
	parser.add_argument("--pretrained-model-path", type=str, default="stories42M.pt")
	parser.add_argument("--max_sentence_len", type=int, default=None)
	parser.add_argument("--seed", type=int, default=1337)
	parser.add_argument("--epochs", type=int, default=5)
	parser.add_argument("--option", type=str,
						help='prompt: the Llama parameters are frozen; finetune: Llama parameters are updated',
						choices=('generate', 'prompt', 'finetune', 'lora'), default="generate")
	parser.add_argument("--use_gpu", action='store_true')
	parser.add_argument("--generated_sentence_low_temp_out", type=str, default="generated-sentence-temp-0.txt")
	parser.add_argument("--generated_sentence_high_temp_out", type=str, default="generated-sentence-temp-1.txt")
	parser.add_argument("--dev_out", type=str, default="cfimdb-dev-prompting-output.txt")
	parser.add_argument("--test_out", type=str, default="cfimdb-test-prompting-output.txt")
	parser.add_argument("--lora_rank", type=int, default=4, help="LoRA rank (r)")
	parser.add_argument("--lora_alpha", type=float, default=1.0, help="LoRA alpha scaling factor")

	# hyper parameters
	parser.add_argument("--batch_size", help='sst: 64, cfimdb: 8 can fit a 12GB GPU', type=int, default=8)
	parser.add_argument("--hidden_dropout_prob", type=float, default=0.3)
	parser.add_argument("--lr", type=float, help="learning rate, default lr for 'pretrain': 1e-3, 'finetune': 1e-5",
						default=2e-5)

	args, _ = parser.parse_known_args()
	print(f"args: {vars(args)}")
	return args
args = get_args()

tokenizer = Tokenizer(args.max_sentence_len)
train_data, num_labels = create_data(args.train, tokenizer, 'train')
train_dataset = LlamaDataset(train_data, args)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=args.batch_size,
								  collate_fn=train_dataset.collate_fn)
for step, batch in enumerate(train_dataloader):
    b_ids, b_labels, b_sents, b_padding_mask = batch['token_ids'], batch['labels'], batch['sents'], batch['padding_mask']
    print(b_padding_mask.sum(dim=1))
    print(b_padding_mask)

/opt/anaconda3/envs/llama_hw/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


args: {'train': 'data/cfimdb-train.txt', 'dev': 'data/cfimdb-dev.txt', 'test': 'data/cfimdb-test.txt', 'label_names': 'data/cfimdb-label-mapping.json', 'pretrained_model_path': 'stories42M.pt', 'max_sentence_len': None, 'seed': 1337, 'epochs': 5, 'option': 'generate', 'use_gpu': False, 'generated_sentence_low_temp_out': 'generated-sentence-temp-0.txt', 'generated_sentence_high_temp_out': 'generated-sentence-temp-1.txt', 'dev_out': 'cfimdb-dev-prompting-output.txt', 'test_out': 'cfimdb-test-prompting-output.txt', 'lora_rank': 4, 'lora_alpha': 1.0, 'batch_size': 8, 'hidden_dropout_prob': 0.3, 'lr': 2e-05}
load 1707 data from data/cfimdb-train.txt
tensor([514, 117, 197, 206, 176, 217, 418,  97])
tensor([[ True,  True,  True,  ...,  True,  True,  True],
        [ True,  True,  True,  ..., False, False, False],
        [ True,  True,  True,  ..., False, False, False],
        ...,
        [ True,  True,  True,  ..., False, False, False],
        [ True,  True,  True,  ..., False, False, Fal

In [ ]:
from run_llama_new import LlamaDataset, create_data
import argparse
from tokenizer import Tokenizer
import torch
import json
from types import SimpleNamespace
from torch.utils.data import DataLoader
from classifier import LlamaZeroShotClassifier
import numpy as np

def get_args():
	parser = argparse.ArgumentParser()
	parser.add_argument("--train", type=str, default="data/cfimdb-train.txt")
	parser.add_argument("--dev", type=str, default="data/cfimdb-dev.txt")
	parser.add_argument("--test", type=str, default="data/cfimdb-test.txt")
	parser.add_argument("--label-names", type=str, default="data/cfimdb-label-mapping.json")
	parser.add_argument("--pretrained-model-path", type=str, default="stories42M.pt")
	parser.add_argument("--max_sentence_len", type=int, default=None)
	parser.add_argument("--seed", type=int, default=1337)
	parser.add_argument("--epochs", type=int, default=5)
	parser.add_argument("--option", type=str,
						help='prompt: the Llama parameters are frozen; finetune: Llama parameters are updated',
						choices=('generate', 'prompt', 'finetune', 'lora'), default="generate")
	parser.add_argument("--use_gpu", action='store_true')
	parser.add_argument("--generated_sentence_low_temp_out", type=str, default="generated-sentence-temp-0.txt")
	parser.add_argument("--generated_sentence_high_temp_out", type=str, default="generated-sentence-temp-1.txt")
	parser.add_argument("--dev_out", type=str, default="cfimdb-dev-prompting-output.txt")
	parser.add_argument("--test_out", type=str, default="cfimdb-test-prompting-output.txt")
	parser.add_argument("--lora_rank", type=int, default=4, help="LoRA rank (r)")
	parser.add_argument("--lora_alpha", type=float, default=1.0, help="LoRA alpha scaling factor")

	# hyper parameters
	parser.add_argument("--batch_size", help='sst: 64, cfimdb: 8 can fit a 12GB GPU', type=int, default=8)
	parser.add_argument("--hidden_dropout_prob", type=float, default=0.3)
	parser.add_argument("--lr", type=float, help="learning rate, default lr for 'pretrain': 1e-3, 'finetune': 1e-5",
						default=2e-5)

	args, _ = parser.parse_known_args()
	print(f"args: {vars(args)}")
	return args
args = get_args()

with torch.no_grad():

	device = torch.device('cuda') if args.use_gpu else torch.device('cpu')
	#### Load data
	# create the data and its corresponding datasets and dataloader
	tokenizer = Tokenizer(args.max_sentence_len)
	label_names = json.load(open(args.label_names, 'r'))
	_, num_labels = create_data(args.train, tokenizer, 'train')

	#### Init model
	config = {'pretrained_model_path': args.pretrained_model_path,
			'label_names': label_names,
			'num_labels': num_labels,
			'data_dir': '.',
			'option': args.option}

	config = SimpleNamespace(**config)

	if len(label_names) == 2:
		label_name_str = " or ".join(label_names)
	else:
		label_name_str = ", ".join(label_names[:-1]) + ", or " + label_names[-1]
	prompt_suffix=f"Is this movie {label_name_str}? This movie is "
	model = LlamaZeroShotClassifier(config, tokenizer, label_names)
	model = model.to(device)

	dev_data = create_data(args.dev, tokenizer, 'valid', eos=False, prompt_suffix=prompt_suffix)
	dev_dataset = LlamaDataset(dev_data, args, eos=False)
	dev_dataloader = DataLoader(dev_dataset, shuffle=False, batch_size=args.batch_size, collate_fn=dev_dataset.collate_fn)

	test_data = create_data(args.test, tokenizer, 'test', eos=False, prompt_suffix=prompt_suffix)
	test_dataset = LlamaDataset(test_data, args, eos=False)
	test_dataloader = DataLoader(test_dataset, shuffle=False, batch_size=args.batch_size, collate_fn=test_dataset.collate_fn)

	model.eval() # switch to eval model, will turn off randomness like dropout
	y_true = []
	y_pred = []
	sents = []
	for step, batch in enumerate(dev_dataloader):
		b_ids, b_labels, b_sents, b_padding_mask = batch['token_ids'], batch['labels'], batch['sents'], batch['padding_mask']

		b_ids = b_ids.to(device)
		b_padding_mask = b_padding_mask.to(device)

		logits = model(b_ids, padding_mask=b_padding_mask)
		logits = logits.detach().cpu().numpy()
		preds = np.argmax(logits, axis=1).flatten()
		print(b_sents[0], b_padding_mask)
		break
print(tokenizer.eos_id)

args: {'train': 'data/cfimdb-train.txt', 'dev': 'data/cfimdb-dev.txt', 'test': 'data/cfimdb-test.txt', 'label_names': 'data/cfimdb-label-mapping.json', 'pretrained_model_path': 'stories42M.pt', 'max_sentence_len': None, 'seed': 1337, 'epochs': 5, 'option': 'generate', 'use_gpu': False, 'generated_sentence_low_temp_out': 'generated-sentence-temp-0.txt', 'generated_sentence_high_temp_out': 'generated-sentence-temp-1.txt', 'dev_out': 'cfimdb-dev-prompting-output.txt', 'test_out': 'cfimdb-test-prompting-output.txt', 'lora_rank': 4, 'lora_alpha': 1.0, 'batch_size': 8, 'hidden_dropout_prob': 0.3, 'lr': 2e-05}
load 1707 data from data/cfimdb-train.txt
load 245 data from data/cfimdb-dev.txt
load 488 data from data/cfimdb-test.txt
tensor([    1,  1670,   525, 29879,  1554, 28893,  1048,   278,  2114,   393,
          263, 14064,  1754,   297, 29871, 29896, 29929, 29941, 29946,   508,
          367,  2343,   322, 26671,  2400,  1432,   323, 11129,   273, 14064,
          393,  5643,   372,  1919

In [3]:
from run_llama_new import LlamaDataset, create_data
import argparse
from tokenizer import Tokenizer
import torch
import json
from types import SimpleNamespace
from torch.utils.data import DataLoader
from classifier import LlamaZeroShotClassifier, LlamaEmbeddingClassifier
import numpy as np

def get_args():
	parser = argparse.ArgumentParser()
	parser.add_argument("--train", type=str, default="data/cfimdb-train.txt")
	parser.add_argument("--dev", type=str, default="data/cfimdb-dev.txt")
	parser.add_argument("--test", type=str, default="data/cfimdb-test.txt")
	parser.add_argument("--label-names", type=str, default="data/cfimdb-label-mapping.json")
	parser.add_argument("--pretrained-model-path", type=str, default="stories42M.pt")
	parser.add_argument("--max_sentence_len", type=int, default=None)
	parser.add_argument("--seed", type=int, default=1337)
	parser.add_argument("--epochs", type=int, default=5)
	parser.add_argument("--option", type=str,
						help='prompt: the Llama parameters are frozen; finetune: Llama parameters are updated',
						choices=('generate', 'prompt', 'finetune', 'lora'), default="generate")
	parser.add_argument("--use_gpu", action='store_true')
	parser.add_argument("--generated_sentence_low_temp_out", type=str, default="generated-sentence-temp-0.txt")
	parser.add_argument("--generated_sentence_high_temp_out", type=str, default="generated-sentence-temp-1.txt")
	parser.add_argument("--dev_out", type=str, default="cfimdb-dev-prompting-output.txt")
	parser.add_argument("--test_out", type=str, default="cfimdb-test-prompting-output.txt")
	parser.add_argument("--lora_rank", type=int, default=4, help="LoRA rank (r)")
	parser.add_argument("--lora_alpha", type=float, default=1.0, help="LoRA alpha scaling factor")

	# hyper parameters
	parser.add_argument("--batch_size", help='sst: 64, cfimdb: 8 can fit a 12GB GPU', type=int, default=8)
	parser.add_argument("--hidden_dropout_prob", type=float, default=0.3)
	parser.add_argument("--lr", type=float, help="learning rate, default lr for 'pretrain': 1e-3, 'finetune': 1e-5",
						default=2e-5)

	args, _ = parser.parse_known_args()
	print(f"args: {vars(args)}")
	return args
args = get_args()

tokenizer = Tokenizer(args.max_sentence_len)
label_names = json.load(open(args.label_names, 'r'))
_, num_labels = create_data(args.train, tokenizer, 'train')

#### Init model
config = {'hidden_dropout_prob': args.hidden_dropout_prob,
          'pretrained_model_path': args.pretrained_model_path,
          'num_labels': num_labels,
          'data_dir': '.',
          'option': 'finetune'}

config = SimpleNamespace(**config)
model = LlamaEmbeddingClassifier(config)

args: {'train': 'data/cfimdb-train.txt', 'dev': 'data/cfimdb-dev.txt', 'test': 'data/cfimdb-test.txt', 'label_names': 'data/cfimdb-label-mapping.json', 'pretrained_model_path': 'stories42M.pt', 'max_sentence_len': None, 'seed': 1337, 'epochs': 5, 'option': 'generate', 'use_gpu': False, 'generated_sentence_low_temp_out': 'generated-sentence-temp-0.txt', 'generated_sentence_high_temp_out': 'generated-sentence-temp-1.txt', 'dev_out': 'cfimdb-dev-prompting-output.txt', 'test_out': 'cfimdb-test-prompting-output.txt', 'lora_rank': 4, 'lora_alpha': 1.0, 'batch_size': 8, 'hidden_dropout_prob': 0.3, 'lr': 2e-05}
load 1707 data from data/cfimdb-train.txt
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
Tr